<a href="https://colab.research.google.com/github/yurimalaquias/SDW2023-ETL/blob/main/SantanderDevWeek2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

**E**xtract

Extraia a lista de IDs de usuário a partir do arquivo CSV. Para cada ID, faça uma requisição GET para obter os dados do usuário correspondente.

In [ ]:
# TODO: Extrair os IDs do Arquivos CSV
import pandas as pd

df = pd.read_csv('SDW2023.csv')
user_ids = df['UserID'].tolist()
print(user_ids)


[4573, 4615, 4666]


In [ ]:
#TODO: Obter os dados de cada ID usando a API da Santander Dev Week 2023
import requests
import json

def get_user(id):
  response = requests.get(f"{sdw2023_api_url}/users/{id}")
  return response.json() if response.status_code == 200 else None

# := operador Walrus (Morça) Esse operador permite que você use e atribua uma variável ao mesmo tempo. Ou seja, você consegue utilizar a variável e atribuir um valor a ela em uma única linha de código.
users = [user for id in user_ids if (user := get_user(id)) is not None] # conceito de compreensão de lista para atribuição do nosso usuario
print(json.dumps(users, indent=2))

[
  {
    "id": 4573,
    "name": "Isa",
    "account": {
      "id": 4850,
      "number": "354119444",
      "agency": "string",
      "balance": 0.0,
      "limit": 0.0
    },
    "card": {
      "id": 4437,
      "number": "121162255",
      "limit": 0.0
    },
    "features": [],
    "news": [
      {
        "id": 8599,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
        "description": "A import\u00e2ncia dos investimentos para Isa: Garantir um futuro financeiro seguro e pr\u00f3spero."
      },
      {
        "id": 8940,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
        "description": ""
      },
      {
        "id": 8941,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
        "description": "Isa, fa\u00e7a seu dinheiro trabalhar para voc\u00ea. Invista com confian\u00e7a! #CrescimentoFinanceiro"
      }

**T**ransform

Utilize a API do OpenAI GPT-4 para gerar uma mensagem de marketing personalizada para cada usuário.

In [ ]:
#!pip install openai

#Estava com erro na hora de executar os comandos e precisei adicionar essas linhas para funcionar.
#!pip install --force-reinstall typing-extensions==4.5
#!pip install --force-reinstall openai==1.8

from google.colab import userdata
openai_api_key = userdata.get('open-api')

In [ ]:
#TODO: Implementar a integração com o ChatGPT usando o modelo GPT-4
import openai
openai.api_key = openai_api_key

def generate_ai_news(user):
  completion = openai.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
          {
              "role": "system",
              "content": "Você é um especialista em marketing bancário."
          },
          {
              "role": "user",
              "content": f"Crie uma mensagem para {user['name']} sobre a importância dos investimentos (máximo de 100 caracteres)"
          },
      ]
  )
  return completion.choices[0].message.content.strip('\"') #strip('\"') para remover as aspas duplas

for user in users:
  news = generate_ai_news(user)
  print(news)
  user['news'].append({
      "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
      "description": news
  })


**L**oad

Atualize a lista de "news" de cada usuário na API com a nova mensagem gerada.

In [ ]:
def update_user(user):
  response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

for user in users:
  success = update_user(user)
  print(f"User {user['name']} updated? {success}!")

User Isa updated? True!
User André Lapontele updated? True!
User Jacques Berman updated? True!
